In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pylab as plt
import cv2
import os
import tensorflow as tf
from glob import glob
from tqdm import tqdm
import time
import tensorflow.keras.backend as K

2024-04-13 07:50:40.279797: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 07:50:40.281165: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 07:50:40.310240: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 07:50:40.310746: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 07:50:40.895033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
model_files = [os.path.join('/home/radhika/strykathon/unet/model', 
                            f'unet_jacc_{class_id}.keras') for class_id in range(1,7)]

classes_and_colors = np.array([[0,0,0] ,[255,0,0], [25,155,187], 
                      [125,255,0], [125,255,255], [175,15,123],
                      [141,255,12] ])

In [3]:
height = 1024//4
width = 1280//4

label = 2
img_path = '/home/radhika/strykathon/unet/dataset/train/frames/frame000_seq_1.png'
label_path = '/home/radhika/strykathon/unet/dataset/train/labels/frame000_seq_1.png'
train_path = '/home/radhika/strykathon/unet/dataset/train'
model_path = os.path.join('/home/radhika/strykathon/unet/model', f'unet_jacc_{label}.keras')
# log_path = os.path.join('./log/', f'log_{label}.csv')
# save_path = './save/'

In [4]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [5]:
def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [6]:
def load_model(model_path : str):
    return tf.keras.models.load_model(model_path, custom_objects={'jaccard_distance': jaccard_distance})


In [74]:
def change_color_of_binary_mask(image , class_id : int):

    return np.array([ [ classes_and_colors[class_id] if pixel == 255 \
            else classes_and_colors[0] for pixel in row ] for row in image ])

In [75]:
image_path = r"/home/radhika/strykathon/unet/dataset/train/labels1/frame000_seq_1.png"
image = cv2.imread(image_path,0)
img = change_color_of_binary_mask(image,1)
cv2.imshow("image",img)
cv2.waitKey(5000)
cv2.destroyAllWindows()

error: OpenCV(4.8.1) /io/opencv/modules/highgui/src/precomp.hpp:155: error: (-215:Assertion failed) src_depth != CV_16F && src_depth != CV_32S in function 'convertToShow'


In [70]:
def get_class_wise_predicted_mask ( x , class_id ) :
    name = x.split('/')[-1]
    
    x = cv2.imread ( x , cv2.IMREAD_COLOR )
    x = cv2.resize ( x , ( width , height ) )
    x = x / 255.0
    x = np.expand_dims ( x , axis = 0 )
    
    model = load_model(model_files[class_id-1])

    p = model.predict(x)[0]
    p = np.squeeze(p, axis=-1)
    # print ( p.shape )
    #return p
    cv2.imwrite("/home/radhika/strykathon/unet/dataset/train/predictions/pred1.png",p)
    # p = cv2.imread("/home/radhika/strykathon/unet/dataset/train/predictions/pred1.png",0)
    return change_color_of_binary_mask( p ,class_id)

In [71]:
predicted_mask = get_class_wise_predicted_mask ( r"/home/radhika/strykathon/unet/dataset/train/frames/frame000_seq_1.png",1 )
# plt.imshow(predicted_mask)
cv2.imshow("random",predicted_mask)
cv2.waitKey(5000)
cv2.destroyAllWindows()

1/1 [==============================] - 1s 668ms/step
(256, 320, 3)


In [14]:
import numpy as np
from functools import reduce

def concatenate_masks( first_mask, second_mask ):
    return [[ second_mask [ row_index ][ col_index ] or pixel \
            for col_index , pixel in enumerate ( row ) ]\
            for row_index , row in enumerate ( first_mask ) ]

def concatenate_masks_rgb(first_mask, second_mask):
    return [[
        [second_mask[row_index][col_index][channel] and pixel[channel]
         for channel in range(3)]  # Iterate over RGB channels
        for col_index, pixel in enumerate(row)]
        for row_index, row in enumerate(first_mask)]

def concatenate_class_wise_masks ( image_path : str, num_of_classes = 6 ):
    image = cv2.imread ( image_path )
    h, w, _ = image.shape

    blank_canvas = np.zeros(( h , w, 3 ))

    predicted_masks = [ get_class_wise_predicted_mask( image_path, class_id ) \
                       for class_id in range ( 1 , num_of_classes + 1 )]  
    
    predicted_masks.append(blank_canvas)
                                
    return np.array(reduce( concatenate_masks_rgb , predicted_masks ))


In [15]:
import cv2
image_path = "/home/radhika/strykathon/unet/dataset/train/frames/frame000_seq_1.png"
final_img = concatenate_class_wise_masks ( image_path, 2 )
cv2.imshow ( "Image", final_img )
cv2.waitKey(5000)
cv2.destroyAllWindows()

1/1 [==============================] - 1s 744ms/step
(256, 320)
1/1 [==============================] - 1s 690ms/step
(256, 320)


In [8]:
l = [1,2,3,4,5]
def add ( x , y ): return x + y
reduce(add,l)


15

In [9]:
def change_color_of_binary_mask(image_path : str, class_id : int):
    image = cv2.imread ( image_path , 0 )
    return np.array([ [ classes_and_colors[class_id] if pixel == 255 \
            else classes_and_colors[0] for pixel in row ] for row in image ])

In [82]:

def change_color_of_binary_mask(image_path: str, class_id : int):
    # Read the image as grayscale
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    mask_rgb = np.stack([mask] * 3, axis=-1)
    
    # Change the color of white pixels in the mask to the specified color
    mask_rgb[mask == 255] = classes_and_colors[ class_id ]
    
    return mask_rgb

In [83]:
import cv2
import numpy as np
from functools import reduce

images = ["/home/radhika/strykathon/unet/dataset/train/labels1/frame000_seq_7.png",
          "/home/radhika/strykathon/unet/dataset/train/labels2/frame000_seq_7.png"]

masks = [ cv2.imread(image,0) for index,image in enumerate(images) ]

masks = [ change_color_of_binary_mask(image,index+1)\
          for index,image in enumerate(images) ]

masks.append ( np.zeros((1024,1280,3)) )
final_image = np.array(reduce ( concatenate_masks_rgb , masks ))

cv2.imshow("dekh kaun",final_image)
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [13]:
l1 = np.array([[0,0,0],[0,0,0]])
l2 = np.array([[11,12,13],[14,2,16]])
[[l2[row_index][col_index] or pixel for col_index,pixel in enumerate(row) ]\
 for row_index,row in enumerate(l1)]

[[11, 12, 13], [14, 2, 16]]